In [5]:
import pandas as pd
import os
import re
from sqlalchemy import create_engine

# --- 1. Import Credentials ---
# Gets the DB credentials from your secret directory.py file
try:
    from directory import DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME

    print("Credentials imported successfully.")
except ImportError:
    print("FATAL ERROR: Could not import database credentials from directory.py")
    print("Please ensure the file exists and the variables (DB_USER, DB_PASS, etc.) are set.")
    raise

# --- 2. Create PostgreSQL Connection ---
# Builds the connection string and the SQLAlchemy engine
db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

try:
    engine = create_engine(db_url)
    print(f"Successfully connected to PostgreSQL database: '{DB_NAME}'")
except Exception as e:
    print("Error connecting to PostgreSQL. Check your credentials and if the server is running.")
    print(f"Error details: {e}")
    raise

# --- 3. Find All Your Clean CSV Files ---
# Finds all .csv files in the main project folder
all_files_in_directory = os.listdir()
clean_csvs = [f for f in all_files_in_directory if f.endswith('.csv')]

if not clean_csvs:
    print("Warning: No clean .csv files were found to upload.")
    print("Make sure you have run your other notebooks to export their data first.")
else:
    print(f"Found {len(clean_csvs)} CSV files to upload: {clean_csvs}")

# --- 4. Loop Through and Upload Each CSV to PostgreSQL ---
for csv_file in clean_csvs:
    try:
        # Create a clean, SQL-safe table name
        # e.g., "Premier_League_2024-25_standings.csv" -> "premier_league_2024_25_standings"
        table_name_base = os.path.splitext(csv_file)[0].lower()
        table_name = re.sub(r'[^a-zA-Z0-9_]', '_', table_name_base)

        print(f"  Uploading: '{csv_file}'  ->  To SQL table: '{table_name}'...")

        # Read the clean CSV into a DataFrame
        df = pd.read_csv(csv_file)

        # Write the DataFrame to the PostgreSQL table
        df.to_sql(table_name, engine, if_exists='replace', index=False)

    except Exception as e:
        print(f"    Error processing {csv_file}: {e}")

print("\nAll clean data has been loaded into PostgreSQL.")

# --- 5. Verify the Upload (Optional) ---
print("\nVerifying tables in your database:")
try:
    query = """
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name; \
            """

    tables_df = pd.read_sql_query(query, engine)
    display(tables_df)

except Exception as e:
    print(f"Error querying database: {e}")

# Close the engine connection
engine.dispose()
print("\nScript complete. Connection closed.")

Credentials imported successfully.
Successfully connected to PostgreSQL database: 'soccer_data'
Found 30 CSV files to upload: ['2024-25_English_FA_Cup,_Semifinals_fixtures.csv', '2024-25_English_FA_Cup,_Final_fixtures.csv', '2024-25_English_FA_Cup,_Third_Round_fixtures.csv', '2024-25_UEFA_Champions_League,_Quarterfinals_fixtures.csv', '2024-25_English_Carabao_Cup,_Second_Round_fixtures.csv', '2024-25_English_Carabao_Cup,_Semifinals_fixtures.csv', '2024-25_English_Carabao_Cup,_Quarterfinals_fixtures.csv', '2024-25_German_Bundesliga_fixtures.csv', '2024-25_English_Carabao_Cup,_Third_Round_fixtures.csv', '2024-25_English_Premier_League_player_stats.csv', '2024-25_UEFA_Champions_League,_Round_of_16_fixtures.csv', '2024-25_English_FA_Cup,_Second_Round_fixtures.csv', '2024-25_English_Premier_League_standings.csv', '2024-25_English_Premier_League_plays_data.csv', '2024-25_UEFA_Champions_League,_League_Phase_fixtures.csv', '2024-25_English_Carabao_Cup,_Fourth_Round_fixtures.csv', '2024-25_UEFA

,table_name
0,2024_25_english_carabao_cup__final_fixtures
1,2024_25_english_carabao_cup__first_round_fixtures
2,2024_25_english_carabao_cup__fourth_round_fixt...
3,2024_25_english_carabao_cup__quarterfinals_fix...
4,2024_25_english_carabao_cup__second_round_fixt...
5,2024_25_english_carabao_cup__semifinals_fixtures
6,2024_25_english_carabao_cup__third_round_fixtures
7,2024_25_english_fa_cup__fifth_round_fixtures
8,2024_25_english_fa_cup__final_fixtures
9,2024_25_english_fa_cup__first_round_fixtures



Script complete. Connection closed.
